## Genetic compression search on LeNet-5 

In [1]:
PATH_PREFIX = '../../'
import sys
sys.path.append(PATH_PREFIX)

In [2]:
import torch
import torch.nn as nn
import pandas as pd
import os
import math

from data.imagenette import ImagenetteDataset
from data.utils.imagenet_utils import *
from utils.weight_sharing import *
from utils.genetic import GeneticController, Individual
from utils.plot import *
from utils.fitness_controller import FitnessController

Setting parameters (for the genetic search settings look in `utils.genetic_config.py`)

In [3]:
# net train settings
BATCH_SIZE = 32
DEVICE = 'cpu'

# net save settings
NET_REPO = 'pytorch/vision:v0.10.0'
NET_TYPE = 'mobilenet_v2'

# dataset settings
DATA_PATH = os.path.join(PATH_PREFIX, 'data/imagenette/')
TOP_ACC = 1

# ga iter count
NUM_GENERATIONS = 36
NUM_PULATION = 12

# ga search settings
CHROMOSOME_RANGES = range(1, 121)

# ga save settings
SAVE_EVOL_FILE = os.path.join(PATH_PREFIX, 'results/test_GA_save.csv')
SAVE_EVERY = 1

# range optimization settings
RANGE_OPTIMIZATION = True
RANGE_OPTIMIZATION_TRESHOLD = 0.97
RANGE_OPTIMIZATION_FILE = os.path.join(PATH_PREFIX, f'models/{NET_TYPE}/saves/{NET_TYPE}_layer_perf.csv')

# target position
TARGET = [1.0, 12.0]
TARGET_LOW_LIMIT = [0.95, 1.0]
LOCK_TARGET = False

# range optimization settings
SHARE_ORDER = None
RETRAIN_AMOUNT = None #[0, 0, 0, 0, 0]
PREC_REDUCT = None # ['f4', 'f4', 'f4', 'f4', 'f4']
CLUST_MOD_FOCUS = None #[5, 5, 5, 5, 5]
CLUST_MOD_SPREAD = None #[0, 0, 0, 0, 0]

Geting somewhat trained LeNet-5

In [4]:
dataset = ImagenetteDataset(BATCH_SIZE, DATA_PATH, val_split=0.5)
model = torch.hub.load(NET_REPO, NET_TYPE, pretrained=True)
model.to(DEVICE)

before_loss = get_accuracy(model, dataset.test_dl, device=DEVICE, topk=TOP_ACC)

Using cache found in /home/coupekv/.cache/torch/hub/pytorch_vision_v0.10.0


Setting weight share controller

In [ ]:
lam_test = lambda : get_accuracy(model, dataset.test_dl, device=DEVICE, topk=TOP_ACC)

ws_controller = WeightShare(model, lam_test)
ws_controller.set_reset()

ws_controller.print_layers_info()

Defining fitness function

In [ ]:
def fitness_vals_fc(individual:Individual):
    # reset the net
    ws_controller.reset()
    
    # share weigts by particle
    if individual.data is None:
        individual.data = ws_controller.share(individual.chromosome, SHARE_ORDER, RETRAIN_AMOUNT, PREC_REDUCT, CLUST_MOD_FOCUS, CLUST_MOD_SPREAD)
    
    return [individual.data['accuracy'], individual.data['compression']]

def fit_from_vals(data, targ_vals):

    return 1 / math.sqrt(pow(1 - (data['accuracy'] - 0.9)/0.1, 2) + pow(1 - (data['compression']/targ_vals[1]), 2))

Defining logging function and elit dealing function

In [ ]:
data = {
    'generation': [],
    'chromosome': [],
    'accuracy': [],
    'accuracy_loss': [],
    'compression': [],
    'share_t': [],
    'train_t': [],
    'acc_t': []
}

data_types = {
    'generation' : 'uint8',
    'accuracy': 'float32',
    'accuracy_loss': 'float32',
    'compression': 'float32',
    'share_t': 'float32',
    'train_t': 'float32',
    'acc_t': 'float32'
}

evol_data = pd.read_csv(SAVE_EVOL_FILE).astype(data_types) if os.path.exists(SAVE_EVOL_FILE) else pd.DataFrame(data).astype(data_types)

def logger_fc(gen_cont:GeneticController):
    global evol_data

    new_data = copy.deepcopy(data)

    for indiv in gen_cont.population:

        new_data['generation'].append(gen_cont.generation)
        new_data['chromosome'].append(indiv.chromosome)
        new_data['accuracy'].append(indiv.data['accuracy'])
        new_data['accuracy_loss'].append(before_loss - indiv.data['accuracy'])
        new_data['compression'].append(indiv.data['compression'])
        new_data['share_t'].append(indiv.data['times']['share'])
        new_data['train_t'].append(indiv.data['times']['train'])
        new_data['acc_t'].append(indiv.data['times']['test'])

    # saving progress
    evol_data = evol_data.append(pd.DataFrame(new_data).astype(data_types))
    if gen_cont.generation % SAVE_EVERY == SAVE_EVERY - 1:
        evol_data.reset_index(drop=True, inplace=True)
        os.makedirs(os.path.dirname(SAVE_EVOL_FILE), exist_ok=True)
        evol_data.to_csv(SAVE_EVOL_FILE, index=False)
    
def deal_elit(population):
    for individual in population:
        if individual.data is not None:
            individual.data['times'] = {
                'share': 0,
                'train': 0,
                'test': 0
            }


Setting ranges with optimization

In [ ]:
lam_test_inp = lambda _ : get_accuracy(model, dataset.test_dl, DEVICE)

if np.array(CHROMOSOME_RANGES).ndim() == 1:
    CHROMOSOME_RANGES = [CHROMOSOME_RANGES for _ in range(len(ws_controller.model_layers))]

if RANGE_OPTIMIZATION:
    CHROMOSOME_RANGES = ws_controller.get_optimized_layer_ranges(CHROMOSOME_RANGES, lam_test_inp, RANGE_OPTIMIZATION_TRESHOLD, 
        savefile=RANGE_OPTIMIZATION_FILE)

for c_range in CHROMOSOME_RANGES:
    print(len(c_range))

Run evolution

In [ ]:
fit_controll = FitnessController(TARGET, fitness_vals_fc, fit_from_vals, target_max_offset=1, lock=LOCK_TARGET, target_limit=TARGET_LOW_LIMIT)
genetic = GeneticController(CHROMOSOME_RANGES, NUM_PULATION, fit_controll)

if evol_data.size != 0:
    genetic.load_from_pd(evol_data, verbose=True)

genetic.run(NUM_GENERATIONS, logger_fc, deal_elit=deal_elit, verbose=True)

See output

In [ ]:
evol_data.tail()

Plotting data

In [ ]:
plot_alcr(evol_data)
plt.title('Genetic Algorithm on LeNet-5')